In [1]:
import os
import sys
sys.path.append('..')

#os.environ["R_HOME"] = os.path.join(os.environ["LOCALAPPDATA"], "Programs", "R", "R-4.3.1")

In [2]:
import rpy2
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr

import pandas as pd
from pyopensus import Opensus

ModuleNotFoundError: No module named 'rpy2'

In [45]:
os.environ["R_HOME"] = r_home

In [27]:
utils = importr("utils")

In [46]:
#utils.install_packages("read.dbc")

In [47]:
full_path = os.path.join(os.environ["HOMEPATH"], "Documents", "data", "SIHSUS", "AIH")
path_to_dbc = os.path.join(full_path, "RDCE0801.dbc")

In [48]:
read_dbc = importr('read.dbc')

In [62]:
rdf = read_dbc.read_dbc(path_to_dbc)
pydf = robjects.conversion.rpy2py(rdf)

In [66]:
read_dbc.dbc2dbf(path_to_dbc, os.path.join(full_path, 'RDCE0801.dbf'))

1


In [64]:
rdf

UF_ZI,ANO_CMPT,MES_CMPT,...,ETNIA,SEQUENCIA,REMESSA
230000,2008,01,...,NA_integer_,2092,HE230...
230000,2008,01,,NA_integer_,2093,HE230...
230000,2008,01,,NA_integer_,2094,HE230...
230000,2008,01,,NA_integer_,2095,HE230...
...,...,...,,...,...,...
231410,2008,01,,NA_integer_,5,HM231...
231410,2008,01,,NA_integer_,6,HM231...
231410,2008,01,,NA_integer_,7,HM231...
231410,2008,01,,NA_integer_,8,HM231...


In [69]:
host = 'ftp.datasus.gov.br'

In [35]:
class Opensus:
    '''
    
    '''
    def __init__(self) -> None:
        self.host = 'ftp.datasus.gov.br'
        self.baseftp = FTP(self.host)
        self.baseftp.login()
        self.basepath = '/dissemin/publicos/'
        
        self.sys_included = {'sihsus': self.basepath+'SIHSUS/200801_/Dados/', 
                             'siasus': self.basepath+'SIASUS/200801_/Dados/'}
        
    def listdir(self, origin=None):
        '''
            ...
        '''
        if origin is None:
            self.baseftp.cwd('/dissemin/publicos/')
            self.baseftp.retrlines('LIST')
        else:
            if origin.lower() in self.sys_included.keys():
                self.baseftp.cwd(self.sys_included[origin.lower()])
                self.baseftp.retrlines('LIST')
                
    def retrieve_year(self, dest, origin, uf, year, mode="RD", verbose=False):
        '''
            ...
        '''
        this_year = dt.date.today().year
        if year < 2008 or year > 2023:
            raise Exception() # --
            
        if origin.lower() not in self.sys_included.keys():
            raise Exception() # --
        else:
            self.baseftp.cwd(self.sys_included[origin.lower()])
            
        year_str = f'{year}'[2:]
        filename = f"{mode}{uf.upper()}{year_str}"
        month_lst = ['01', '02', '03', '04', '05', '06', '07',
                     '08', '09', '10', '11', '12']
            
        for cur_month in month_lst:
            filename_ = f'{filename}{cur_month}.dbc'
            if verbose:
                print(f'Download do arquivo {filename_} ...', end='')
                
            with open(os.path.join(dest, filename_), 'wb') as fp:
                self.baseftp.retrbinary(f'RETR {filename_}', fp.write)
                
            if verbose:
                print(' Feito.')
                
        
        

In [70]:
obj = Opensus()

In [37]:
#obj.listdir(origin='SIHSUS')

In [5]:
dest = os.path.join(os.environ['HOMEPATH'], 'Documents', 'data', 'opendatasus', 'sihsus')

#for year in range(2012, 2024):
#    obj.retrieve_year(dest, 'sihsus', 'ce', year, mode='RD', verbose=True)

In [6]:
#obj.baseftp.login()

In [47]:
ftplib.all_errors

(ftplib.Error, OSError, EOFError, ssl.SSLError)

In [7]:
#from ftplib import OSError

In [60]:
ex = FTP(host)

In [61]:
ex.login()

'230 User logged in.'

In [4]:
#obj.baseftp.pwd()

In [ ]:
from rpy2.robjects.packages import importr

In [ ]:
import rpy2.robjects as robjects

In [2]:
import os

In [67]:
from simpledbf import Dbf5

PyTables is not installed. No support for HDF output.


In [70]:
df = Dbf5(os.path.join(full_path, 'RDCE0801.dbf')).to_dataframe()

In [71]:
df

,UF_ZI,ANO_CMPT,MES_CMPT,ESPEC,CGC_HOSP,N_AIH,IDENT,CEP,MUNIC_RES,NASC,...,CID_ASSO,CID_MORTE,COMPLEX,FINANC,FAEC_TP,REGCT,RACA_COR,ETNIA,SEQUENCIA,REMESSA
0,230000,2008,01,03,NaN,2308100006069,1,63430000,230540,19280315,...,NaN,NaN,02,06,NaN,0000,01,NaN,2092,HE23000001N200801.DTS
1,230000,2008,01,03,NaN,2308100006080,1,63430000,230540,19820117,...,NaN,NaN,02,06,NaN,0000,03,NaN,2093,HE23000001N200801.DTS
2,230000,2008,01,03,NaN,2308100006102,1,63430000,230540,19790802,...,NaN,NaN,02,06,NaN,0000,03,NaN,2094,HE23000001N200801.DTS
3,230000,2008,01,03,NaN,2308100006146,1,63430000,230540,19290401,...,NaN,NaN,02,06,NaN,0000,01,NaN,2095,HE23000001N200801.DTS
4,230000,2008,01,03,NaN,2308100006168,1,63430000,230540,19210321,...,NaN,NaN,02,06,NaN,0000,01,NaN,2096,HE23000001N200801.DTS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29818,231410,2008,01,01,NaN,2308102140839,1,62300000,231410,19520211,...,NaN,NaN,02,06,NaN,0000,03,NaN,4,HM23141001N200801.DTS
29819,231410,2008,01,01,NaN,2308102140840,1,62300000,231410,19760911,...,NaN,NaN,02,06,NaN,0000,03,NaN,5,HM23141001N200801.DTS
29820,231410,2008,01,01,NaN,2308102140850,1,62300000,231410,19760720,...,NaN,NaN,02,06,NaN,0000,03,NaN,6,HM23141001N200801.DTS
29821,231410,2008,01,01,NaN,2308102140861,1,62300000,231410,20080109,...,NaN,NaN,02,06,NaN,0000,03,NaN,7,HM23141001N200801.DTS


In [7]:
os.environ["R_HOME"] = rpy2.situation.get_r_home()

AttributeError: module 'rpy2' has no attribute 'situation'

In [14]:
os.listdir(os.path.join(os.environ["PROGRAMFILES"], "R"))

['R-4.3.1']

In [12]:
os.environ

environ{'ALLUSERSPROFILE': 'C:\\ProgramData',
        'APPDATA': 'C:\\Users\\Higor Monteiro\\AppData\\Roaming',
        'COMMONPROGRAMFILES': 'C:\\Program Files\\Common Files',
        'COMMONPROGRAMFILES(X86)': 'C:\\Program Files (x86)\\Common Files',
        'COMMONPROGRAMW6432': 'C:\\Program Files\\Common Files',
        'COMPUTERNAME': 'DESKTOP-8TTQVR5',
        'COMSPEC': 'C:\\WINDOWS\\system32\\cmd.exe',
        'CONDA_DEFAULT_ENV': 'mywork',
        'CONDA_EXE': 'C:\\Users\\Higor Monteiro\\anaconda3\\Scripts\\conda.exe',
        'CONDA_PREFIX': 'C:\\Users\\Higor Monteiro\\anaconda3\\envs\\mywork',
        'CONDA_PREFIX_1': 'C:\\Users\\Higor Monteiro\\anaconda3',
        'CONDA_PROMPT_MODIFIER': '(mywork) ',
        'CONDA_PYTHON_EXE': 'C:\\Users\\Higor Monteiro\\anaconda3\\python.exe',
        'CONDA_SHLVL': '2',
        'DRIVERDATA': 'C:\\Windows\\System32\\Drivers\\DriverData',
        'EFC_6752': '1',
        'FPS_BROWSER_APP_PROFILE_STRING': 'Internet Explorer',
        'FPS